# Cycling Wardrobe 
The goal of this project is to create and use a twitter bot that will tweet once per day with a summary of weather forecast for a city (London in my case) and a recommendation of what to wear for a cycling commute based on the weather forecast.

## Weather Forecast
This part of the notebook takes care of the Weather forecast and the connection with OpenWeather

In [ ]:
import requests
import configparser
from datetime import datetime, timedelta

In [ ]:
def parse_config(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    return config

In [ ]:
## Open Weather functions
## TODO: Find rain probability

def time_slot_datapoint(openweather_response, day_to_predict, starting_timings):
    for datapoint in openweather_response['list']:
        if datetime.fromtimestamp(datapoint['dt']).date()==day_to_predict and datetime.fromtimestamp(datapoint['dt']).hour in starting_timings:
            return(datapoint)

def time_datapoint(openweather_response, day_to_predict, time):
    # The hours that will fit a 3-hour window at the given time are t-2, t-1 and t
    return time_slot_datapoint(openweather_response, day_to_predict, [time-2, time-1, time])
  
def extract_timeslot_weather_data(datapoint):
    weather = {}
    weather['dt'] = datetime.fromtimestamp(datapoint['dt'])
    weather['min_temperature'] = datapoint['main']['temp_min']
    weather['max_temperature'] = datapoint['main']['temp_max']
    weather['humidity'] = datapoint['main']['humidity']
    weather['wind'] = datapoint['wind']['speed']
    return weather
    
## TODO add 7am and 6pm as variables
# It returns weather forecast data for today (if ran before 7am) or for tomorrow otherwise
def weather_data(weather_api_key):
    ## Sunrise / Sunset data (note: these times are computed for the day when the call is made)
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    city = 'London,uk'
    request = base_url+'?q={}&appid={}&units=metric'.format(city, weather_api_key)
    data = requests.get(request).json()
    day_weather = {}
    day_weather['sunrise'] = datetime.fromtimestamp(data['sys']['sunrise'])
    day_weather['sunset'] = datetime.fromtimestamp(data['sys']['sunset'])
    
    ## Forecast 3-hour windows data
    base_url = 'http://api.openweathermap.org/data/2.5/forecast'
    city = 'London,uk'
    request = base_url+'?q={}&appid={}&units=metric'.format(city, weather_api_key)
    data = requests.get(request).json()
    
    # Commuting times
    morning_time = 7
    afternoon_time = 18
    
    now = datetime.now()
    # If we run this after the morning time, we will predict for tomorrow
    if now.hour>morning_time:
        day_to_predict = (now + timedelta(days = 1)).date()
    else:
        day_to_predict = now.date()
    
    weather = {}
    weather['morning'] = {'sunrise': day_weather['sunrise']}
    weather['morning'].update(extract_timeslot_weather_data(time_datapoint(data, day_to_predict, morning_time)))
    weather['afternoon'] = {'sunset': day_weather['sunset']}
    weather['afternoon'].update(extract_timeslot_weather_data(time_datapoint(data, day_to_predict, afternoon_time)))
    
    return weather

## Recommendation
Depending on the weather factors, the system will recommend a specific type of clothes and cycling gear

In [ ]:
## First iteration based only on the lowest of min temperature and returning a string
def gear_recommendation(weather_forecast):
    min_temperature = min(weather_forecast['morning']['min_temperature'], 
                          weather_forecast['afternoon']['min_temperature'])
    
    if min_temperature < 5:
        return ("Wear jacket with underskin, thick globes, long bib and thermal socks. Cover for head and neck.")
    elif min_temperature < 10:
        return ("Wear jacket, thick globes and long bib. Cover for head and neck.")
    elif min_temperature < 15:
        return ("Wear long mallot or mallot with arm warmers, long globes and long bib or short with leg warmers. Cover for neck.")
    elif min_temperature < 20:
        return ("Wear mallot (carry arm warmers in case), fingerless globes and short bib.")
    else:
        return ("Wear thin mallot, fingerless globes and short bib.")
    

## Twitter integration
This part of the notebook deals with the twitter API and the posts that will be shared in social media

In [ ]:
import tweepy
from tweepy import OAuthHandler

In [ ]:
def authorise_twitter_api(config):
    auth = OAuthHandler(config['DEFAULT']['consumer_key'], 
                        config['DEFAULT']['consumer_secret'])
    auth.set_access_token(config['DEFAULT']['access_token'], 
                          config['DEFAULT']['access_secret'])
    return auth
    
# TODO: Add emojis
def tweet_recommendation(weather_forecast, recommendation):
    message = 'Weather for {} in London\n'.format(weather_forecast['morning']['dt'].strftime("%A %d"))
    message+= 'Morning - {} degrees (C) with {} kph wind. '.format(weather_forecast['morning']['min_temperature'],
                                                                   weather_forecast['morning']['wind'])
    message+= 'Afternoon - {} degrees with {} kph wind.\n'.format(weather_forecast['afternoon']['min_temperature'],
                                                                  weather_forecast['afternoon']['wind'])
    
    message += recommendation
#     api.update_status("This is my first tweet. Hello World!")
    print(message)

## Execution
This part of the notebook glues everything together by getting the weather forecasts, making a recommendation for what to wear and tweeting it

In [ ]:
## Intitialisation
config = parse_config('config.cfg')
weather_api_key = config['DEFAULT']['open_weather_key']
auth = authorise_twitter_api(config)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
weather_forecast = weather_data(weather_api_key)
weather_forecast

In [ ]:
# ## Weather
weather_forecast = weather_data(weather_api_key)

# ## Recommendation
clothes = gear_recommendation(weather_forecast)

## Tweet
tweet_recommendation(weather_forecast, clothes)